In [1]:
import json
import datetime,time
import pytz
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from statistics import mean
import statsmodels.api as sm
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from statistics import mean
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsRegressor

In [12]:
path = "tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", \
        "tweets_#nfl.txt", "tweets_#patriots.txt", \
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
topics = ["gohawks", "gopatriots", "nfl", "patriots", "sb49", "superbowl"]
test_files_1 =["sample0_period1.txt","sample0_period2.txt","sample0_period3.txt"]
test_files_2 =["sample1_period1.txt","sample1_period2.txt","sample1_period3.txt"]
test_files_3 =["sample2_period1.txt","sample2_period2.txt","sample2_period3.txt"]

for i, fl in enumerate(files):
    print("files[" + str(i) + "] => " + fl)

files[0] => tweets_#gohawks.txt
files[1] => tweets_#gopatriots.txt
files[2] => tweets_#nfl.txt
files[3] => tweets_#patriots.txt
files[4] => tweets_#sb49.txt
files[5] => tweets_#superbowl.txt


In [3]:
def generate_df(pddata_raw):
    """
    Create a new dataframe with desired form
    """
    df = pd.DataFrame([],columns=['time unit','tweets','retweets','followers sum','followers max','hr_min'])
    
    col = pddata_raw.columns.get_loc('time')
    df['time unit'] = range(int(pddata_raw.iloc[len(pddata_raw.index)-1,col] - pddata_raw.iloc[0,col]+1))
    df['tweets'] = pddata_raw.groupby("time")['tweets'].sum()
    df['retweets'] = pddata_raw.groupby("time")['retweets'].sum()
    df['followers sum'] = pddata_raw.groupby("time")['followers'].sum()
    df['followers max'] = pddata_raw.groupby("time")["followers"].max()
    df['hr_min'] = pddata_raw.groupby("time")['hr_min'].mean()        
    # reset index of df
    df = df.fillna(0).reset_index(drop=True)
    
    return df

In [4]:

def parse(path,files):
    """
    Parse x conponent of the dataset into pandas DataFrame including columns of:
    tweets, retweets, total_followers, max_followers, mentioned, media, active, author, favourites_count, title
    with lines of hours,
    where mentioned: number of @ in tweets per hour
          media: number of url in tweets per hour
          active: a measure of active state of author
          author: number of unique authors post tweet per hour
          favourites_count: the total number of likes by this user
          title: length of this tweet's title
    Parse y of dataset as number of tweets in the next hour.
    """
    start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
    end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))   

    start_hour_idx = ((2-1)*31+1-14)*24+8
    end_hour_idx = ((2-1)*31+1-14)*24+20
    start_minute_idx = (((2-1)*31+1-14)*24 + (8-0))*12 + 0//5    
        
    # extract raw features
    data_raw = [[],[],[]]
    for file in files:
        for line in open(path + file, 'r', encoding="utf-8") :
            row_tmp = []
            a = json.loads(line)
            citation_date = a['citation_date']
            tweet = 1
            retweet = a['metrics']['citations']['total']
            foll = a['author']['followers']
            hr_min=1

            # append to row_tmp
            row_tmp.append(citation_date)        
            row_tmp.append(tweet)        
            row_tmp.append(retweet)
            row_tmp.append(foll)  
            row_tmp.append(hr_min) 
            
            data_raw[0].append(row_tmp)            

    # generate raw pandas dataframe
    pddata_raw_1 = transfer_time(data_raw[0],'hour')
    pddata_raw_1['time'] = pddata_raw_1['time'] - pddata_raw_1.loc[0,'time']
    
    
    # generate df and df_y for each time slot
    df_1 = generate_df(pddata_raw_1)  
    df_y_1 = df_1.iloc[1:,1].reset_index(drop=True)
    df_1 = df_1[:len(df_y_1)]
    #print('2',df_1)
    
    
    return (df_1.iloc[:,1:],df_y_1)

In [37]:
(df_test1,df_y_test1),(df_2_test1,df_y_2_test1),(df_3_test1,df_y_3_test1) = parse_dataset(path,test_files_1)

start_time= 1422806400.0 date= 2015-02-01 08:00:00-08:00
citation_date= 1422709237 date= 2015-01-31 05:00:37-08:00
citation_date= 1422709279 date= 2015-01-31 05:01:19-08:00
citation_date= 1422709683 date= 2015-01-31 05:08:03-08:00
citation_date= 1422709802 date= 2015-01-31 05:10:02-08:00
citation_date= 1422709818 date= 2015-01-31 05:10:18-08:00
citation_date= 1422711645 date= 2015-01-31 05:40:45-08:00
citation_date= 1422711973 date= 2015-01-31 05:46:13-08:00
citation_date= 1422712506 date= 2015-01-31 05:55:06-08:00
citation_date= 1422712717 date= 2015-01-31 05:58:37-08:00
citation_date= 1422712809 date= 2015-01-31 06:00:09-08:00
citation_date= 1422713053 date= 2015-01-31 06:04:13-08:00
citation_date= 1422713125 date= 2015-01-31 06:05:25-08:00
citation_date= 1422713336 date= 2015-01-31 06:08:56-08:00
citation_date= 1422713567 date= 2015-01-31 06:12:47-08:00
citation_date= 1422713658 date= 2015-01-31 06:14:18-08:00
citation_date= 1422713764 date= 2015-01-31 06:16:04-08:00
citation_date= 

citation_date= 1422717013 date= 2015-01-31 07:10:13-08:00
citation_date= 1422717023 date= 2015-01-31 07:10:23-08:00
citation_date= 1422717049 date= 2015-01-31 07:10:49-08:00
citation_date= 1422717128 date= 2015-01-31 07:12:08-08:00
citation_date= 1422717154 date= 2015-01-31 07:12:34-08:00
citation_date= 1422717172 date= 2015-01-31 07:12:52-08:00
citation_date= 1422717240 date= 2015-01-31 07:14:00-08:00
citation_date= 1422717242 date= 2015-01-31 07:14:02-08:00
citation_date= 1422717248 date= 2015-01-31 07:14:08-08:00
citation_date= 1422717300 date= 2015-01-31 07:15:00-08:00
citation_date= 1422717335 date= 2015-01-31 07:15:35-08:00
citation_date= 1422717722 date= 2015-01-31 07:22:02-08:00
citation_date= 1422717921 date= 2015-01-31 07:25:21-08:00
citation_date= 1422717942 date= 2015-01-31 07:25:42-08:00
citation_date= 1422718025 date= 2015-01-31 07:27:05-08:00
citation_date= 1422718135 date= 2015-01-31 07:28:55-08:00
citation_date= 1422718197 date= 2015-01-31 07:29:57-08:00
citation_date=

citation_date= 1422725831 date= 2015-01-31 09:37:11-08:00
citation_date= 1422725842 date= 2015-01-31 09:37:22-08:00
citation_date= 1422725904 date= 2015-01-31 09:38:24-08:00
citation_date= 1422726023 date= 2015-01-31 09:40:23-08:00
citation_date= 1422726075 date= 2015-01-31 09:41:15-08:00
citation_date= 1422726110 date= 2015-01-31 09:41:50-08:00
citation_date= 1422726307 date= 2015-01-31 09:45:07-08:00
citation_date= 1422726405 date= 2015-01-31 09:46:45-08:00
citation_date= 1422726549 date= 2015-01-31 09:49:09-08:00
citation_date= 1422726605 date= 2015-01-31 09:50:05-08:00
citation_date= 1422726639 date= 2015-01-31 09:50:39-08:00
citation_date= 1422726680 date= 2015-01-31 09:51:20-08:00
citation_date= 1422726827 date= 2015-01-31 09:53:47-08:00
citation_date= 1422726835 date= 2015-01-31 09:53:55-08:00
citation_date= 1422726914 date= 2015-01-31 09:55:14-08:00
citation_date= 1422726917 date= 2015-01-31 09:55:17-08:00
citation_date= 1422726920 date= 2015-01-31 09:55:20-08:00
citation_date=

citation_date= 1422846079 date= 2015-02-01 19:01:19-08:00
citation_date= 1422846080 date= 2015-02-01 19:01:20-08:00
citation_date= 1422846081 date= 2015-02-01 19:01:21-08:00
citation_date= 1422846082 date= 2015-02-01 19:01:22-08:00
citation_date= 1422846085 date= 2015-02-01 19:01:25-08:00
citation_date= 1422846087 date= 2015-02-01 19:01:27-08:00
citation_date= 1422846088 date= 2015-02-01 19:01:28-08:00
citation_date= 1422846088 date= 2015-02-01 19:01:28-08:00
citation_date= 1422846091 date= 2015-02-01 19:01:31-08:00
citation_date= 1422846092 date= 2015-02-01 19:01:32-08:00
citation_date= 1422846093 date= 2015-02-01 19:01:33-08:00
citation_date= 1422846094 date= 2015-02-01 19:01:34-08:00
citation_date= 1422846096 date= 2015-02-01 19:01:36-08:00
citation_date= 1422846098 date= 2015-02-01 19:01:38-08:00
citation_date= 1422846099 date= 2015-02-01 19:01:39-08:00
citation_date= 1422846101 date= 2015-02-01 19:01:41-08:00
citation_date= 1422846101 date= 2015-02-01 19:01:41-08:00
citation_date=

citation_date= 1422846527 date= 2015-02-01 19:08:47-08:00
citation_date= 1422846528 date= 2015-02-01 19:08:48-08:00
citation_date= 1422846528 date= 2015-02-01 19:08:48-08:00
citation_date= 1422846529 date= 2015-02-01 19:08:49-08:00
citation_date= 1422846530 date= 2015-02-01 19:08:50-08:00
citation_date= 1422846531 date= 2015-02-01 19:08:51-08:00
citation_date= 1422846532 date= 2015-02-01 19:08:52-08:00
citation_date= 1422846533 date= 2015-02-01 19:08:53-08:00
citation_date= 1422846533 date= 2015-02-01 19:08:53-08:00
citation_date= 1422846533 date= 2015-02-01 19:08:53-08:00
citation_date= 1422846537 date= 2015-02-01 19:08:57-08:00
citation_date= 1422846539 date= 2015-02-01 19:08:59-08:00
citation_date= 1422846540 date= 2015-02-01 19:09:00-08:00
citation_date= 1422846543 date= 2015-02-01 19:09:03-08:00
citation_date= 1422846543 date= 2015-02-01 19:09:03-08:00
citation_date= 1422846547 date= 2015-02-01 19:09:07-08:00
citation_date= 1422846548 date= 2015-02-01 19:09:08-08:00
citation_date=

...

citation_date= 1422846532 date= 2015-02-01 19:08:52-08:00
citation_date= 1422846532 date= 2015-02-01 19:08:52-08:00
citation_date= 1422846534 date= 2015-02-01 19:08:54-08:00
citation_date= 1422846534 date= 2015-02-01 19:08:54-08:00
citation_date= 1422846535 date= 2015-02-01 19:08:55-08:00
citation_date= 1422846535 date= 2015-02-01 19:08:55-08:00
citation_date= 1422846536 date= 2015-02-01 19:08:56-08:00
citation_date= 1422846536 date= 2015-02-01 19:08:56-08:00
citation_date= 1422846536 date= 2015-02-01 19:08:56-08:00
citation_date= 1422846538 date= 2015-02-01 19:08:58-08:00
citation_date= 1422846538 date= 2015-02-01 19:08:58-08:00
citation_date= 1422846538 date= 2015-02-01 19:08:58-08:00
citation_date= 1422846539 date= 2015-02-01 19:08:59-08:00
citation_date= 1422846539 date= 2015-02-01 19:08:59-08:00
citation_date= 1422846540 date= 2015-02-01 19:09:00-08:00
citation_date= 1422846540 date= 2015-02-01 19:09:00-08:00
citation_date= 1422846541 date= 2015-02-01 19:09:01-08:00
citation_date=

citation_date= 1422846976 date= 2015-02-01 19:16:16-08:00
citation_date= 1422846981 date= 2015-02-01 19:16:21-08:00
citation_date= 1422846982 date= 2015-02-01 19:16:22-08:00
citation_date= 1422846985 date= 2015-02-01 19:16:25-08:00
citation_date= 1422846986 date= 2015-02-01 19:16:26-08:00
citation_date= 1422846988 date= 2015-02-01 19:16:28-08:00
citation_date= 1422846992 date= 2015-02-01 19:16:32-08:00
citation_date= 1422846994 date= 2015-02-01 19:16:34-08:00
citation_date= 1422846995 date= 2015-02-01 19:16:35-08:00
citation_date= 1422846996 date= 2015-02-01 19:16:36-08:00
citation_date= 1422846999 date= 2015-02-01 19:16:39-08:00
citation_date= 1422847000 date= 2015-02-01 19:16:40-08:00
citation_date= 1422847002 date= 2015-02-01 19:16:42-08:00
citation_date= 1422847004 date= 2015-02-01 19:16:44-08:00
citation_date= 1422847006 date= 2015-02-01 19:16:46-08:00
citation_date= 1422847007 date= 2015-02-01 19:16:47-08:00
citation_date= 1422847008 date= 2015-02-01 19:16:48-08:00
citation_date=

citation_date= 1422847556 date= 2015-02-01 19:25:56-08:00
citation_date= 1422847559 date= 2015-02-01 19:25:59-08:00
citation_date= 1422847561 date= 2015-02-01 19:26:01-08:00
citation_date= 1422847563 date= 2015-02-01 19:26:03-08:00
citation_date= 1422847563 date= 2015-02-01 19:26:03-08:00
citation_date= 1422847565 date= 2015-02-01 19:26:05-08:00
citation_date= 1422847571 date= 2015-02-01 19:26:11-08:00
citation_date= 1422847600 date= 2015-02-01 19:26:40-08:00
citation_date= 1422847576 date= 2015-02-01 19:26:16-08:00
citation_date= 1422847577 date= 2015-02-01 19:26:17-08:00
citation_date= 1422847583 date= 2015-02-01 19:26:23-08:00
citation_date= 1422847584 date= 2015-02-01 19:26:24-08:00
citation_date= 1422847584 date= 2015-02-01 19:26:24-08:00
citation_date= 1422847586 date= 2015-02-01 19:26:26-08:00
citation_date= 1422847588 date= 2015-02-01 19:26:28-08:00
citation_date= 1422847589 date= 2015-02-01 19:26:29-08:00
citation_date= 1422847602 date= 2015-02-01 19:26:42-08:00
citation_date=

citation_date= 1422846122 date= 2015-02-01 19:02:02-08:00
citation_date= 1422846128 date= 2015-02-01 19:02:08-08:00
citation_date= 1422846132 date= 2015-02-01 19:02:12-08:00
citation_date= 1422846144 date= 2015-02-01 19:02:24-08:00
citation_date= 1422846145 date= 2015-02-01 19:02:25-08:00
citation_date= 1422846145 date= 2015-02-01 19:02:25-08:00
citation_date= 1422846153 date= 2015-02-01 19:02:33-08:00
citation_date= 1422846154 date= 2015-02-01 19:02:34-08:00
citation_date= 1422846154 date= 2015-02-01 19:02:34-08:00
citation_date= 1422846158 date= 2015-02-01 19:02:38-08:00
citation_date= 1422846164 date= 2015-02-01 19:02:44-08:00
citation_date= 1422846167 date= 2015-02-01 19:02:47-08:00
citation_date= 1422846168 date= 2015-02-01 19:02:48-08:00
citation_date= 1422846169 date= 2015-02-01 19:02:49-08:00
citation_date= 1422846174 date= 2015-02-01 19:02:54-08:00
citation_date= 1422846176 date= 2015-02-01 19:02:56-08:00
citation_date= 1422846185 date= 2015-02-01 19:03:05-08:00
citation_date=

citation_date= 1423053942 date= 2015-02-04 04:45:42-08:00
citation_date= 1423053968 date= 2015-02-04 04:46:08-08:00
citation_date= 1423054169 date= 2015-02-04 04:49:29-08:00
citation_date= 1423054423 date= 2015-02-04 04:53:43-08:00
citation_date= 1423054444 date= 2015-02-04 04:54:04-08:00
citation_date= 1423054457 date= 2015-02-04 04:54:17-08:00
citation_date= 1423054536 date= 2015-02-04 04:55:36-08:00
citation_date= 1423054688 date= 2015-02-04 04:58:08-08:00
citation_date= 1423054693 date= 2015-02-04 04:58:13-08:00
citation_date= 1423054761 date= 2015-02-04 04:59:21-08:00
citation_date= 1423054898 date= 2015-02-04 05:01:38-08:00
citation_date= 1423055089 date= 2015-02-04 05:04:49-08:00
citation_date= 1423055110 date= 2015-02-04 05:05:10-08:00
citation_date= 1423055197 date= 2015-02-04 05:06:37-08:00
citation_date= 1423055259 date= 2015-02-04 05:07:39-08:00
citation_date= 1423055290 date= 2015-02-04 05:08:10-08:00
citation_date= 1423055318 date= 2015-02-04 05:08:38-08:00
citation_date=

In [38]:
df_test1.head

<bound method NDFrame.head of    tweets  retweets  followers sum  followers max  hr_min
0      52       109       424498.0       168371.0       5
1      79       761      2975692.0      2034387.0       6
2      94       226       860594.0       328882.0       7
3     101       258      2349147.0       368626.0       8
4     122       483      1369748.0       291130.0       9>

In [39]:
df_y_test1

0     79
1     94
2    101
3    122
4    120
Name: tweets, dtype: int64

In [50]:

def parse_dataset(path,files):
    """
    Parse x conponent of the dataset into pandas DataFrame including columns of:
    tweets, retweets, total_followers, max_followers, mentioned, media, active, author, favourites_count, title
    with lines of hours,
    where mentioned: number of @ in tweets per hour
          media: number of url in tweets per hour
          active: a measure of active state of author
          author: number of unique authors post tweet per hour
          favourites_count: the total number of likes by this user
          title: length of this tweet's title
    Parse y of dataset as number of tweets in the next hour.
    """
    start_time = time.mktime(time.strptime("2015-02-01 08:00:00",'%Y-%m-%d %H:%M:%S'))
    end_time = time.mktime(time.strptime("2015-02-01 20:00:00",'%Y-%m-%d %H:%M:%S'))   

    start_hour_idx = ((2-1)*31+1-14)*24+8
    end_hour_idx = ((2-1)*31+1-14)*24+20
    start_minute_idx = (((2-1)*31+1-14)*24 + (8-0))*12 + 0//5    
        
    # extract raw features
    pst_tz = pytz.timezone('America/Los_Angeles')
    p = datetime.datetime.fromtimestamp(start_time, pst_tz) 
#     print("start_time=",start_time,"date=",p)
    data_raw = [[],[],[]]
    for file in files:
        for line in open(path + file, 'r', encoding="utf-8") :
            row_tmp = []
            a = json.loads(line)
            citation_date = a['citation_date']
            tweet = 1
            retweet = a['metrics']['citations']['total']
            foll = a['author']['followers']
            hr_min=1
            p = datetime.datetime.fromtimestamp(citation_date, pst_tz) 
#             print("citation_date=",citation_date,"date=",p)
            # append to row_tmp
            row_tmp.append(citation_date)        
            row_tmp.append(tweet)        
            row_tmp.append(retweet)
            row_tmp.append(foll)  
            row_tmp.append(hr_min) 
            # assign to 3 periods
            if citation_date < start_time:
                data_raw[0].append(row_tmp)
            elif citation_date < end_time:
                data_raw[1].append(row_tmp)
            else:
                data_raw[2].append(row_tmp)            
#         print("next file")
    # generate raw pandas dataframe
    pddata_raw_1 = transfer_time(data_raw[0],'hour')
    pddata_raw_1['time'] = pddata_raw_1['time'] - pddata_raw_1.loc[0,'time']
    
    pddata_raw_2 = transfer_time(data_raw[1],'minute')
#     pddata_raw_2['time'] = pddata_raw_2['time'] - start_minute_idx
    pddata_raw_2['time'] = pddata_raw_2['time'] - pddata_raw_2.loc[0,'time']

    pddata_raw_3 = transfer_time(data_raw[2],'hour')
#     pddata_raw_3['time'] = pddata_raw_3['time'] - end_hour_idx - 1    
    pddata_raw_3['time'] = pddata_raw_3['time'] - pddata_raw_3.loc[0,'time']
    #print('1',pddata_raw_1)
    
    # generate df and df_y for each time slot
    df_1 = generate_df(pddata_raw_1)  
    df_y_1 = df_1.iloc[1:,1].reset_index(drop=True)
    df_1 = df_1[:len(df_y_1)]
    #print('2',df_1)
    
    df_2 = generate_df(pddata_raw_2)
    df_y_2 = df_2.iloc[1:,1].reset_index(drop=True)
    df_2 = df_2[:len(df_y_2)]
   
    df_3 = generate_df(pddata_raw_3)
#     print(df_3)
    df_y_3 = df_3.iloc[1:,1].reset_index(drop=True)
    df_3 = df_3[:len(df_y_3)]
    
    return (df_1.iloc[:,1:],df_y_1), (df_2.iloc[:,1:],df_y_2), (df_3.iloc[:,1:],df_y_3)

In [34]:
def transfer_time(data_raw,time_type):
    
    pst_tz = pytz.timezone('America/Los_Angeles')
    
    # sort according to time
    pddata_raw = pd.DataFrame(data_raw,columns=['time','tweets','retweets','followers','hr_min'])
    pddata_raw = pddata_raw.sort_values(by = 'time')
    pddata_raw = pddata_raw.reset_index(drop=True)               

    # calculate hour index and minute index from time
    if time_type == 'hour':
        hour_accu = []
        hour_min = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)  
            hour_accu.append(((p.month-1)*31+p.day)*24+p.hour)  
            hour_min.append(p.hour)
        pddata_raw['time'] = hour_accu
        pddata_raw['hr_min'] = hour_min
    elif time_type == 'minute':
        minu_accu = []
        hour_min = []
        for index, row in pddata_raw.iterrows():  
            p = datetime.datetime.fromtimestamp(row['time'], pst_tz)                    
            minu_accu.append((((p.month-1)*31+p.day)*24 + (p.hour-0))*12 + p.minute//5)   
            hour_min.append(p.minute//5)
        pddata_raw['time'] = minu_accu    
        pddata_raw['hr_min'] = hour_min
    else:
        print("Invalid time type")
        
    return pddata_raw

In [40]:
def six_times_window(df,df_y):
    m=df.shape[0]
    df_new= pd.DataFrame(np.zeros((m-4,5)),columns=['tweets','retweets','followers sum','followers max',\
                                  'hr_min'])
    df_y_new= pd.DataFrame(np.zeros((m-4)))
    for i in range(m-4):
        
        for j in range(5):
            df_new.iloc[i,j]=(df.iloc[i,j]+df.iloc[i+1,j]+df.iloc[i+2,j]+df.iloc[i+3,j]+df.iloc[i+4,j])
        j=3
        df_new.iloc[i,3]=np.max([df.iloc[i,j],df.iloc[i+1,j],df.iloc[i+2,j],df.iloc[i+3,j],df.iloc[i+4,j]])
        df_y_new.iloc[i,0]=df_y.iloc[i+4]
    
    return df_new, df_y_new    

In [41]:
def plot_recipe(df_y, pred_y):
    """
    This function plots fitted values vs true values
    """
    plt.figure()
    area = np.pi * (4)**2/4
    plt.scatter(df_y, pred_y, s = area)
    plt.plot([df_y.min(), df_y.max()], [df_y.min(), df_y.max()], 'k--', lw = 1)
    plt.xlabel('true values')
    plt.ylabel('fitted values')
    plt.show()

In [44]:
(dfo,dfo_y),(dfo_2,dfo_y_2),(dfo_3,dfo_y_3) =parse_dataset(path,files)

In [9]:
df, df_y  =six_times_window(dfo,dfo_y)
df_2, df_y_2  =six_times_window(dfo_2,dfo_y_2)
df_3, df_y_3  =six_times_window(dfo_3,dfo_y_3)

In [49]:
(df_test1,df_y_test1),(df_2_test1,df_y_2_test1),(df_3_test1,df_y_3_test1) = parse_dataset(path,test_files_1)
(df_test2,df_y_test2),(df_2_test2,df_y_2_test2),(df_3_test2,df_y_3_test2) = parse_dataset(path,test_files_2)
(df_test3,df_y_test3),(df_2_test3,df_y_2_test3),(df_3_test3,df_y_3_test3) = parse_dataset(path,test_files_3)

   time unit  tweets  retweets  followers sum  followers max  hr_min
0          0      59        74      3564378.0      3329958.0       0
1          1      48       103       146905.0        59219.0       1
2          2      94        99        99613.0        16753.0       2
3          3      45        61       119754.0        27669.0       3
4          4      77       141       505238.0       257549.0       4
5          5      87       131       164136.0        27199.0       5
   time unit  tweets  retweets  followers sum  followers max  hr_min
0          0      58      1285     10709633.0      9677129.0      20
1          1      87        98      1846822.0      1458675.0      21
2          2      43        43        48886.0         5022.0      22
3          3      27        30        73474.0        32613.0      23
4          4      44        62       155066.0        36269.0       0
5          5      46        56       102647.0        13628.0       1
   time unit  tweets  retweets  fo

In [47]:
df_3_test1

,tweets,retweets,followers sum,followers max,hr_min
0,59,74,3564378.0,3329958.0,0
1,48,103,146905.0,59219.0,1
2,94,99,99613.0,16753.0,2
3,45,61,119754.0,27669.0,3
4,77,141,505238.0,257549.0,4


In [23]:
df_test1

,tweets,retweets,followers sum,followers max,hr_min
0,52,109,424498.0,168371.0,5
1,79,761,2975692.0,2034387.0,6
2,94,226,860594.0,328882.0,7
3,101,258,2349147.0,368626.0,8
4,122,483,1369748.0,291130.0,9


In [12]:
df_test1, df_y_test1  =six_times_window(df_test1,df_y_test1)
df_2_test1, df_y_2_test1  =six_times_window(df_2_test1,df_y_2_test1)
df_3_test1, df_y_3_test1  =six_times_window(df_3_test1,df_y_3_test1)
df_test2, df_y_test2  =six_times_window(df_test2,df_y_test2)
df_2_test2, df_y_2_test2  =six_times_window(df_2_test2,df_y_2_test2)
df_3_test2, df_y_3_test2  =six_times_window(df_3_test2,df_y_3_test2)
df_test3, df_y_test3  =six_times_window(df_test3,df_y_test3)
df_2_test3, df_y_2_test3  =six_times_window(df_2_test3,df_y_2_test3)
df_3_test3, df_y_3_test3  =six_times_window(df_3_test3,df_y_3_test3)

# RandomForest

In [14]:
reg = RandomForestRegressor(oob_score=True,n_estimators=200,max_depth=20,min_samples_leaf=2,min_samples_split=10, bootstrap=True,max_features='sqrt',random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
pred1=reg.predict(df_test1)
pred2=reg.predict(df_test2)
pred3=reg.predict(df_test3)
print(df_y_test1,df_y_test2,df_y_test3)
print(pred1,pred2,pred3)

       0
0  120.0        0
0  846.0       0
0  61.0
[340.77519001] [638.33634891] [202.79927484]


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [15]:
reg = RandomForestRegressor(oob_score=True,n_estimators=1200,max_depth=20,min_samples_leaf=4,min_samples_split=2, bootstrap=True,max_features='sqrt',random_state=42)
reg.fit(df_2,df_y_2)
pred = reg.predict(df_2)
pred1=reg.predict(df_2_test1)
pred2=reg.predict(df_2_test2)
pred3=reg.predict(df_2_test3)
print(df_y_2_test1,df_y_2_test2,df_y_2_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


        0
0  1123.0        0
0  903.0       0
0  28.0
[5062.22864342] [1085.21591104] [1042.27265207]


In [16]:
reg = RandomForestRegressor(oob_score=True,n_estimators=200,max_depth=20,min_samples_leaf=4,min_samples_split=2, bootstrap=True,max_features='sqrt',random_state=42)
reg.fit(df_3,df_y_3)
pred = reg.predict(df_3)
pred1=reg.predict(df_3_test1)
pred2=reg.predict(df_3_test2)
pred3=reg.predict(df_3_test3)
print(df_y_3_test1,df_y_3_test2,df_y_3_test3)
print(pred1,pred2,pred3)

      0
0  87.0       0
0  46.0       0
0  43.0
[103.95607076] [199.11456449] [203.31185616]


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


# GradientBoostingRegressor

In [17]:
reg = GradientBoostingRegressor(n_estimators=200,max_depth=20,min_samples_leaf=2,min_samples_split=10, max_features='sqrt',random_state=42)
reg.fit(df,df_y)
pred = reg.predict(df)
pred1=reg.predict(df_test1)
pred2=reg.predict(df_test2)
pred3=reg.predict(df_test3)
print(df_y_test1,df_y_test2,df_y_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


       0
0  120.0        0
0  846.0       0
0  61.0
[298.04358475] [573.50261936] [177.73571437]


In [18]:
reg = GradientBoostingRegressor(n_estimators=1200,max_depth=20,min_samples_leaf=4,min_samples_split=2, max_features='sqrt',random_state=42)
reg.fit(df_2,df_y_2)
pred = reg.predict(df_2)
pred1=reg.predict(df_2_test1)
pred2=reg.predict(df_2_test2)
pred3=reg.predict(df_2_test3)
print(df_y_2_test1,df_y_2_test2,df_y_2_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


        0
0  1123.0        0
0  903.0       0
0  28.0
[6816.75807019] [1016.80809268] [965.41975729]


In [19]:
reg = GradientBoostingRegressor(n_estimators=200,max_depth=20,min_samples_leaf=4,min_samples_split=2, max_features='sqrt',random_state=42)
reg.fit(df_3,df_y_3)
pred = reg.predict(df_3)
pred1=reg.predict(df_3_test1)
pred2=reg.predict(df_3_test2)
pred3=reg.predict(df_3_test3)
print(df_y_3_test1,df_y_3_test2,df_y_3_test3)
print(pred1,pred2,pred3)

      0
0  87.0       0
0  46.0       0
0  43.0
[30.03838049] [11.88626886] [30.80890984]


F:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Neural Network

In [20]:
scaler = StandardScaler()
scaler.fit(df)
df_trans=scaler.transform(df)
reg = MLPRegressor(hidden_layer_sizes=(50,600,300,),activation='relu', solver='adam', alpha=1e-5, random_state=42) 
reg.fit(df_trans,df_y)
pred = reg.predict(df_trans)
print("MSE=",mean_squared_error(df_y, pred))
pred1=reg.predict(scaler.transform(df_test1))
pred2=reg.predict(scaler.transform(df_test2))
pred3=reg.predict(scaler.transform(df_test3))
print(df_y_test1,df_y_test2,df_y_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE= 3988798.5055075316
       0
0  120.0        0
0  846.0       0
0  61.0
[138.64449719] [597.74290231] [294.78384788]


F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [21]:
scaler = StandardScaler()
scaler.fit(df_2)
df_2_trans=scaler.transform(df_2)
reg = MLPRegressor(hidden_layer_sizes=(600,600,600,),activation='relu', solver='adam', alpha=1e-5, random_state=42) 
reg.fit(df_2_trans,df_y_2)
pred = reg.predict(df_2_trans)
print("MSE=",mean_squared_error(df_y_2, pred))
pred1=reg.predict(scaler.transform(df_2_test1))
pred2=reg.predict(scaler.transform(df_2_test2))
pred3=reg.predict(scaler.transform(df_2_test3))
print(df_y_2_test1,df_y_2_test2,df_y_2_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE= 33264948.38992299
        0
0  1123.0        0
0  903.0       0
0  28.0
[6774.20236825] [6051.60427666] [6946.44812688]


F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [23]:
scaler = StandardScaler()
scaler.fit(df_3)
df_3_trans=scaler.transform(df_3)
reg = MLPRegressor(hidden_layer_sizes=(600,600,600,),activation='relu', solver='adam', alpha=1e-5, random_state=42) 
reg.fit(df_3_trans,df_y_3)
pred = reg.predict(df_3_trans)
print("MSE=",mean_squared_error(df_y_3, pred))
pred1=reg.predict(scaler.transform(df_3_test1))
pred2=reg.predict(scaler.transform(df_3_test2))
pred3=reg.predict(scaler.transform(df_3_test3))
print(df_y_3_test1,df_y_3_test2,df_y_3_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:1316: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MSE= 183911.8928080028
      0
0  87.0       0
0  46.0       0
0  43.0
[574.67856497] [362.31850104] [402.85187849]


F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


# One hour window

In [10]:
(df,df_y) =parse(path,files)

In [17]:
(df_test1,df_y_test1),(df_2_test1,df_y_2_test1),(df_3_test1,df_y_3_test1) = parse_dataset(path,test_files_1)
(df_test2,df_y_test2),(df_2_test2,df_y_2_test2),(df_3_test2,df_y_3_test2) = parse_dataset(path,test_files_2)
(df_test3,df_y_test3),(df_2_test3,df_y_2_test3),(df_3_test3,df_y_3_test3) = parse_dataset(path,test_files_3)

In [21]:
print(df[0:10])

   tweets  retweets  followers sum  followers max  hr_min
0     111       714       223405.0        41818.0       0
1      89       663       187317.0        19558.0       1
2     110       640      1657090.0      1362401.0       2
3     100       267       165256.0        19558.0       3
4     137       468       444858.0        65150.0       4
5     169       346       683834.0       104412.0       5
6     215       580       993090.0       105750.0       6
7     353      1054      1694989.0       174922.0       7
8     569      3224      5685647.0      1300862.0       8
9     533      1982      1838262.0       358231.0       9


In [25]:
scaler = StandardScaler()
scaler.fit(df)
df_trans=scaler.transform(df)
reg = MLPRegressor(hidden_layer_sizes=(1000,500,),activation='relu', solver='adam', alpha=1e-5, random_state=42) 
reg.fit(df_trans,df_y)
pred = reg.predict(df_trans)
print("MSE=",mean_squared_error(df_y, pred))
pred1=reg.predict(scaler.transform(df_test1))
pred2=reg.predict(scaler.transform(df_test2))
pred3=reg.predict(scaler.transform(df_test3))
print(df_y_test1,df_y_test2,df_y_test3)
print(pred1,pred2,pred3)

F:\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  


MSE= 227225011.81404865
0     79
1     94
2    101
3    122
4    120
Name: tweets, dtype: int64 0    180
1    202
2    294
3    555
4    846
Name: tweets, dtype: int64 0    141
1    102
2    144
3    104
4     61
Name: tweets, dtype: int64
[1295.16476579  893.464805   1277.00836324 1280.87307155 1323.39037753] [1332.17968331 1175.67514193 1089.52026958 1259.4663115  1150.08626702] [1007.36753313 1494.733364   1650.35316788 1682.30798545 1533.97456696]


F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.
